# Step 1: upload structure

In [ ]:
from common.structure import StructureUploadWidget
from IPython.display import display
from aiidalab_widgets_base import CodeDropdown
from IPython.display import display
import ipywidgets as ipw

geo_opt_widget = ipw.HTML()

number_of_nodes = ipw.IntText(
    value=1,
    step=1,
    description = "that will be run on",
    disabled=False,
    layout=ipw.Layout(width="210px"),
    style={"description_width":"120px"},
)

btn_submit_cell = ipw.Button(
    description='Run Cell Opt',
    disabled=False,
)
code_group = CodeDropdown(input_plugin='cp2k', text="Select the code")

submit_out = ipw.Output()

class MyStructureUploadWidget(StructureUploadWidget):
    def __init__(self):
        super(MyStructureUploadWidget, self).__init__(node_class='StructureData')

    def _on_click_store(self, change):
        super(MyStructureUploadWidget, self)._on_click_store(change)

        btn_submit_cell.on_click(on_click_submit_cell)
        cell_opt_widget = ipw.HTML("""<h2><strong>Step 2: optimize unit cell</strong></h2>""")
        display(cell_opt_widget,
                ipw.HBox(
                    [
                        code_group,
                        number_of_nodes,
                        ipw.HTML("nodes")
                    ]
                ),
                btn_submit_cell
               )
        display(geo_opt_widget, submit_out)

structure_widget = MyStructureUploadWidget()
display(structure_widget)


In [ ]:
from aiida.orm import load_node, Code
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.upf import get_pseudos_from_structure
from aiida.orm.utils import CalculationFactory
from aiida.work.run import submit, run

In [ ]:
def setup_calc():
    options =  {
        'max_wallclock_seconds': 3600*2,
        'resources':{
            'num_machines': number_of_nodes.value,
        }
    }

    if code_group.selected_code is None:
        print ("Please select a code")
        return None

    if structure_widget.structure_node is None:
        print ("Please select a structure")
        return None

    parameters = {
        'GLOBAL': {
            'RUN_TYPE': 'CELL_OPT',
        },
        'FORCE_EVAL': {
            'METHOD': 'Quickstep',
            'STRESS_TENSOR': 'Analytical',
            'DFT': {
                'BASIS_SET_FILE_NAME': 'BASIS_MOLOPT',
                'POTENTIAL_FILE_NAME': 'POTENTIAL',
                'SCF': {
                    'MAX_SCF': 15,
                    'EPS_SCF': 1e-7,
                    'OT': {
                        'PRECONDITIONER': 'FULL_SINGLE_INVERSE',
                        'MINIMIZER': 'CG'
                    },
                    'OUTER_SCF': {
                        'MAX_SCF': 4,
                        'EPS_SCF': 1e-7,
                    },
                },
                'QS': {
                    'EPS_DEFAULT': 1.0e-12,
                    'WF_INTERPOLATION': 'ps',
                    'EXTRAPOLATION_ORDER': 3,
                },
                'MGRID': {
                    'NGRIDS': 4,
                    'CUTOFF': 280,
                    'REL_CUTOFF': 30,
                },
                'XC': {
                    'XC_FUNCTIONAL': {
                        '_': 'PBE',
                    },
                },
            },
            'SUBSYS': {
                'KIND': [
                    {
                        '_': e ,
                        'BASIS_SET': 'DZVP-MOLOPT-SR-GTH',
                        'POTENTIAL': 'GTH-PBE'
                    } for e in structure_widget.structure_node.get_kind_names()
                ],
            },
        }
    }

    inputs = {
        'code': code_group.selected_code,
        'structure': structure_widget.structure_node,
        'parameters': ParameterData(dict=parameters),
        'settings': ParameterData(dict={}),
        '_options': options,
    }
    return inputs
Cp2kCalculation = CalculationFactory('cp2k')

In [ ]:
optimized_structure = None
btn_submit_eos = ipw.Button(
    description='Submit EOS',
    disabled=False,
)
def on_click_submit_cell(b):
    global optimized_structure
    with submit_out:
        process = Cp2kCalculation.process()
        inputs = setup_calc()
        if inputs is None:
            pass
        else:
            geo_opt_widget.value = """<center>
<i class="fa fa-spinner fa-pulse" style="color:#337ab7;font-size:20em;" ></i>
<br>
<font size="+2"><blink>Running cell optimization...</blink></font>
</center>"""
            btn_submit_cell.disabled = False

            calculation = run(process, **inputs)
            geo_opt_widget.value = """<center>
<i class="fa fa-check" style="color:#337ab7;font-size:20em;" ></i>
<br>
<font size="+2"><blink>Cell optimization is completed!</blink></font>
</center>"""
            eos_widget = ipw.HTML("""<h2><strong>Step 3: Compute Equation of State</strong></h2>""")

            btn_submit_eos.on_click(on_click_submit_eos)
            display(eos_widget,
                    btn_submit_eos,
                    eos_calc_message,
                    eos_calc_output)

In [ ]:
import nglview
from aiida.work.workchain import WorkChain, ToContext, while_, Outputs
from aiida.orm.data.base import Int, Float
from aiida.orm.data.structure import StructureData
from aiida.work.workfunction import workfunction

%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
@workfunction
def scale_structure(structure, scaling_factor):
    """
    Workfunction to scale a structure

    :param structure: An AiiDA structure to scale
    :param scaling_factor: The scaling factor
    :return: The scaled StructureData object
    """
    ase_old = structure.get_ase()
    ase_new = ase_old.copy()
    ase_new.set_cell(ase_old.get_cell() * float(scaling_factor), scale_atoms=True)
    structure = StructureData(ase=ase_new)

    return structure

In [ ]:
class EquationOfState(WorkChain):
    """
    Workchain that for a given structure will compute the equation of state by
    computing the total energy for a set of derived structures with a scaled
    lattice parameter
    """

    @classmethod
    def define(cls, spec):
        """
        This is the most important method of a Workchain, that defines the
        inputs that it takes, the logic of the execution and the outputs
        that are generated in the process 
        """
        super(EquationOfState, cls).define(spec)
        
        # First we define the inputs, specifying the type we expect
        spec.input("structure", valid_type=StructureData)
        spec.input("code", valid_type=Code)
        spec.input("npoints", valid_type=Int)
        
        # The outline describes the business logic that defines
        # which steps are executed in what order and based on
        # what conditions. Each `cls.method` is implemented below
        spec.outline(
            cls.init,
            while_(cls.should_run_cp2k)(
                cls.run_cp2k,
                cls.parse_cp2k,
            ),
            cls.return_result,
        )
        
        # Here we define the output the Workchain will generate and
        # return. Dynamic output allows a variety of AiiDA data nodes
        # to be returned
        spec.dynamic_output()

    def init(self):
        """
        Initialize variables and the scales we want to compute
        """
        npoints = self.inputs.npoints.value
        self.ctx.i = 0
        self.ctx.cell_volume = 0.0
        self.ctx.scales = sorted([1 - pow(-1, x)*0.02*int((x+1)/2) for x in range(npoints)])
        self.ctx.result = []
        self.ctx.options = {
            'max_wallclock_seconds': 3600*2,
            'resources':{
                'num_machines': number_of_nodes.value,
            }
        }

        self.ctx.parameters = {
            'GLOBAL': {
                'RUN_TYPE': 'ENERGY',
            },
            'FORCE_EVAL': {
                'METHOD': 'Quickstep',
                'DFT': {
                    'BASIS_SET_FILE_NAME': 'BASIS_MOLOPT',
                    'POTENTIAL_FILE_NAME': 'POTENTIAL',
                    'SCF': {
                        'MAX_SCF': 15,
                        'EPS_SCF': 1e-7,
                        'OT': {
                            'PRECONDITIONER': 'FULL_SINGLE_INVERSE',
                            'MINIMIZER': 'CG'
                        },
                        'OUTER_SCF': {
                            'MAX_SCF': 4,
                            'EPS_SCF': 1e-7,
                        },
                    },
                    'QS': {
                        'EPS_DEFAULT': 1.0e-12,
                        'WF_INTERPOLATION': 'ps',
                        'EXTRAPOLATION_ORDER': 3,
                    },
                    'MGRID': {
                        'NGRIDS': 4,
                        'CUTOFF': 280,
                        'REL_CUTOFF': 30,
                    },
                    'XC': {
                        'XC_FUNCTIONAL': {
                            '_': 'PBE',
                        },
                    },
                },
                'SUBSYS': {
                    'KIND': [
                        {
                            '_': e,
                            'BASIS_SET': 'DZVP-MOLOPT-SR-GTH',
                            'POTENTIAL': 'GTH-PBE'
                        } for e in optimized_structure.get_kind_names()
                    ],
                },
            }
        }       


    def should_run_cp2k(self):
        """
        This is the main condition of the while loop, as defined
        in the outline of the Workchain. We only run another
        pw.x calculation if the current iteration is smaller than
        the total number of scale factors we want to compute
        """
        return self.ctx.i < len(self.ctx.scales)

    def run_cp2k(self):
        """
        This is the main function that will perform a pw.x
        calculation for the current scaling factor
        """
        scale = self.ctx.scales[self.ctx.i]
        structure = scale_structure(self.inputs.structure, Float(scale))
        self.ctx.i += 1
        self.ctx.cell_volume = structure.get_cell_volume()

        # Create the input dictionary
        inputs = {
            'code'       : self.inputs.code,
            'structure'  : structure,
            'parameters' : ParameterData(dict=self.ctx.parameters),
            '_options'   : self.ctx.options,
        }

        # Create the calculation process and launch it
        self.report("Running pw.x for the scale factor {}".format(scale))
        process = Cp2kCalculation.process()
        future  = submit(process, **inputs)

        return ToContext(pw=Outputs(future))

    def parse_cp2k(self):
        """
        Extract the volume and total energy of the last completed PwCalculation
        """
        volume = self.ctx.cell_volume
        energy = self.ctx.pw["output_parameters"].dict.energy
        self.ctx.result.append((volume, energy))
        
        self.plot_data()

    def return_result(self):
        """
        Attach the results of the PwCalculations and the initial structure to the outputs
        """
        result = {
            "initial_structure": self.inputs.structure,
            "result": ParameterData(dict={"eos": self.ctx.result}),
        }

        for link_name, node in result.iteritems():
            self.out(link_name, node)

        self.report("Workchain <{}> completed successfully".format(self.calc.pk))

        return

    def plot_data(self):
        global ax, fig
        ax.plot(*zip(*self.ctx.result), marker='o', linestyle='--', color='r')
        fig.canvas.draw()


In [ ]:
npoints = 5

In [ ]:
eos_calc_output = ipw.Output()
eos_calc_message = ipw.HTML('')
ax = None
fig = None

def on_click_submit_eos(b):
    global ax, fig
    global optimized_structure

    # Initialize plot variables
    fig, ax = plt.subplots(1,1)
    garbage = ax.set_xlabel(u"Volume [Å^3]")
    garbage = ax.set_ylabel(u"Total energy [eV]")

    btn_submit_eos.disabled = True
    eos_calc_message.value = """<h4>Please wait, the Equation of State is being computed
    <i class="fa fa-spinner fa-pulse" style="color:#337ab7;" ></i></h4>
    Final results of the Equation of State workchain:
    """
    with eos_calc_output:
        outputs = run(
            EquationOfState,
            npoints=Int(npoints),
            structure=optimized_structure,
            code=code_group.selected_code,
        )
        print ("{volume:12}  {energy:12}".format(volume="Volume (A^3)", energy="Energy (eV)"))
        print ("{}".format("-"*26))
        for p in outputs["result"].get_dict()['eos']:
            print ("{volume:>12.5f}  {energy:>12.5f}".format(volume=p[0], energy=p[1]))
    eos_calc_message.value = '<h4>The Equation of State is computed <i class="fa fa-check " style="color:#337ab7;" ></i></h4>'
    btn_submit_eos.disabled = False